In [0]:
%run ./CommonUtils

In [0]:
mountPoint = get_mnt('silver')
mountPoint

In [0]:
entity_name = dbutils.widgets.get("entity_name").lower()

In [0]:
df = spark.read.table(f"{get_catalog_name(entity_name,'bronze')}.{entity_name.split('_inc')[0]}")

In [0]:
# df = add_audit_columns(df,ingestion_layer="silver",entity_name=entity_name)
if "lastUpdateDate" in df.columns:
    df = df.withColumnRenamed("lastUpdateDate","sourceLastUpdateDate")

In [0]:
table_exist_check = spark.sql(f"show tables in {get_catalog_name(entity_name,'silver')}").select('tableName').filter(f"tableName = '{entity_name.split('_inc')[0]}'").count() == 1
if table_exist_check:
    print('table exists')
else:
    dbutils.notebook.run('./Create Or Replace Tables',300,{"ingestion_layer":"silver","entity_name":entity_name})

In [0]:
generate_clean_file(mountPoint,entity_name,df,ingestion_layer="silver")

In [0]:
create_or_replace_table(mountPoint,entity_name, ingestion_layer="silver")

In [0]:
print("Silver Ingestion Completed")